In [3]:
import numpy as np
import spiceypy
import pandas as pd

In [4]:
filename = r"/global/cfs/projectdirs/m4895/RAE2Data/interpolatedRAE2MasterFile.csv"

rawData = pd.read_csv(filename)
data = rawData
data['time'] = pd.to_datetime(data['time'])
data.drop('Unnamed: 0',axis=1,inplace=True)
data.set_index('time',inplace=True)
start_date = pd.to_datetime("1974-01-01 14:00")
end_date = pd.to_datetime("1974-01-01 16:00")

# Selecting rows within the date range
df = data[(data.index >= start_date) & (data.index <= end_date)].copy()

In [5]:
df.head()

,frequency_band,position_x,position_y,position_z,earth_unit_vector_x,earth_unit_vector_y,earth_unit_vector_z,right_ascension,declination,rv1_coarse,rv2_coarse,rv1_fine,rv2_fine,rv_temp
time,,,,,,,,,,,,,,
1974-01-01 14:00:00.472,8,1405.177612,-2324.264282,676.282288,-0.984300,-0.102829,-0.143458,19.990051,22.859650,1.558787e+06,678791.3750,0.000,0.0000,1092909793
1974-01-01 14:00:02.397,8,1407.012268,-2323.617737,674.645844,-0.984299,-0.102834,-0.143460,19.992851,22.825348,1.024165e+06,572116.1250,0.000,0.0000,1092909793
1974-01-01 14:00:04.322,8,1408.846924,-2322.971191,673.009399,-0.984298,-0.102838,-0.143462,19.995651,22.791046,2.062520e+06,780709.0625,935338.625,579085.3125,1092909793
1974-01-01 14:00:06.247,8,1410.681580,-2322.324646,671.372955,-0.984297,-0.102843,-0.143464,19.998451,22.756744,1.355128e+06,572116.1250,0.000,0.0000,1092909793
1974-01-01 14:00:08.172,8,1412.516235,-2321.678101,669.736511,-0.984297,-0.102848,-0.143466,20.001251,22.722443,1.793041e+06,637875.3125,0.000,0.0000,1092909793


In [ ]:
# Use first timestamp from filtered CSV slice to drive SPICE lookups (per instructions)
spiceypy.kclear()
# TODO: update to your meta-kernel that loads lunar/earth ephemerides before running
spiceypy.furnsh("/path/to/your/meta_kernel.tm")
sample_time = df.index[0].to_pydatetime()
et = spiceypy.utc2et(sample_time.strftime("%Y-%m-%dT%H:%M:%S"))
state, _ = spiceypy.spkezr('EARTH', et, 'B1950', 'NONE', 'MOON')
earth_from_moon = np.array(state[:3])
earth_unit_vec = earth_from_moon / np.linalg.norm(earth_from_moon)
print(f"UTC sample epoch: {sample_time}")
print(f"Earth unit vector in lunicentric B1950 frame: {earth_unit_vec}")
